In [ ]:
# 1. single approach that predicts N edges using max_balance in one run
# and then predict using lowrank method

# 2. iterative approach that predicts N edges in X runs
# and then predict using LR method


# result:
# plot of two lines: 
# x-axis: N, minimum triangle count threshold (similar to embeddedness)
# y-axis: accuracy